# Conversational RAG agents

https://python.langchain.com/docs/tutorials/qa_chat_history/

## Basic chain with history

### Import data

This should probably be tested, maybe better to also add the question to the page_content

In [13]:
import json
from langchain.schema import Document

def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

documents = prepare_qa_documents("../data/home0001qa.json")
print(documents[:5])

[Document(metadata={'question': 'Do i own my 0001 home outright?'}, page_content='When you buy a 0001 home, you own the title in the traditional way. If you need, we’ll help you find the right mortgage and can recommend real estate lawyers. You keep full legal ownership of your home, with the added benefit that you can spend time in other locations whenever you want.'), Document(metadata={'question': 'Does furniture come included?'}, page_content='Yes, each new home comes fully furnished and equipped so that you can move in easily with just your suitcase. Each interior is designed for flexibility and functionality and we work directly with designers from our community to source pieces often straight from their studios.'), Document(metadata={'question': 'Can i change the design of my home?'}, page_content="Legally you own your home and are free to do what you want with it. However, to maintain access to home0001's network in other locations, your home does need to meet our standards and

### LLM setup

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### Create embeddings, vectorstore, retriever

In [15]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# Will be replaced in a bit
retriever = vectorstore.as_retriever()

### Incorporate basic retriever into a question-answering chain

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "What is Home0001?"})
print(response["answer"])

Home0001 is a global housing network that offers fully-equipped and furnished homes, allowing individuals to move in with just their suitcase and swap cities whenever they like. It functions as a distributed housing collective where homeowners can access homes in other cities for free, with only a cleaning fee charged each time. Home0001 also includes community dinners and events for its members.


### Add chat history

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# constructs a chain that accepts keys input and chat_history as input, 
# and has the same output schema as a retriever.
# It manages the case where chat_history is empty, and otherwise applies prompt
# | llm | StrOutputParser() | retriever in sequence.
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [19]:
print(contextualize_q_prompt)

input_variables=['chat_history', 'input'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messag

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# generate a question_answer_chain, with input keys context, chat_history, and input
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# applies the history_aware_retriever and question_answer_chain in sequence, retaining intermediate outputs such as the retrieved context for convenience. 
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "Hi i'm Che Guevara. What is Home0001?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)
print(ai_msg_1["answer"])

second_question = "is it a timeshare? btw what's my name?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=second_question),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)

print(ai_msg_2["answer"])

Home0001 is a global housing network and distributed housing collective that offers fully-equipped and furnished homes. Members can move in with just their suitcase and have access to homes in other cities for free, only paying a cleaning fee. It promotes community living through events and dinners while allowing homeowners to experience living in various locations.
No, Home0001 is not a timeshare; it offers traditional home ownership where you own the title to your individual home. As for your name, you mentioned you are Che Guevara.


In [22]:
print(chat_history)

[HumanMessage(content="Hi i'm Che Guevara. What is Home0001?", additional_kwargs={}, response_metadata={}), AIMessage(content='Home0001 is a global housing network and distributed housing collective that offers fully-equipped and furnished homes. Members can move in with just their suitcase and have access to homes in other cities for free, only paying a cleaning fee. It promotes community living through events and dinners while allowing homeowners to experience living in various locations.', additional_kwargs={}, response_metadata={}), HumanMessage(content="is it a timeshare? btw what's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='No, Home0001 is not a timeshare; it offers traditional home ownership where you own the title to your individual home. As for your name, you mentioned you are Che Guevara.', additional_kwargs={}, response_metadata={})]


## Stateful management of chat history w/ LangGraph

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [25]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict
### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "hi my name is Flippy, what is home001?"},
    config=config,
)
print(result["answer"])

Home0001 is a global housing network that offers fully-equipped and furnished homes for individuals to move into with just their suitcase. It allows members to swap cities whenever they like and provides access to homes in other locations for free, with only a cleaning fee charged. The concept promotes flexible living and community engagement through events and dinners.


In [27]:
result = app.invoke(
    {"input": "swap where? what's my name dawg?"},
    config=config,
)
print(result["answer"])

You can swap to any location offered within the home0001 network that closely matches the value of your current home. Your name is Flippy.


In [28]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

hi my name is Flippy, what is home001?
================================== Ai Message ==================================

Home0001 is a global housing network that offers fully-equipped and furnished homes for individuals to move into with just their suitcase. It allows members to swap cities whenever they like and provides access to homes in other locations for free, with only a cleaning fee charged. The concept promotes flexible living and community engagement through events and dinners.
================================ Human Message =================================

swap where? what's my name dawg?
================================== Ai Message ==================================

You can swap to any location offered within the home0001 network that closely matches the value of your current home. Your name is Flippy.


In [29]:
result = app.invoke(
    {"input": "can i buy in spain?"},
    config=config,
)
print(result["answer"])

Yes, you can buy a home0001 in Spain, as the process for buying a home is the same regardless of your location. There are no extra costs such as stamp duty or additional taxes for non-US citizens.


In [30]:
result = app.invoke(
    {"input": "no but i mean do you currently have locations availabe there?"},
    config=config,
)
print(result["answer"])

Currently, home0001 does not have locations available in Spain. The available locations include Los Angeles, New York City, Paris, Berlin, London, Mexico City, and some rural areas.


In [31]:
result = app.invoke(
    {"input": "which rural areas specifically?"},
    config=config,
)
print(result["answer"])

The specific rural areas where home0001 is available have not been detailed in the provided context. For more information, you would need to contact home0001 directly to inquire about the exact rural locations.


## Agents

Agents generate the input to the retriever directly, without necessarily needing us to explicitly build in contextualization, as we did above;  

Agents can execute multiple retrieval steps in service of a query, or refrain from executing a retrieval step altogether (e.g., in response to a generic greeting from a user).


In [2]:
import json
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

documents = prepare_qa_documents("../data/home0001qa.json")
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

In [3]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "FAQ_retriever",
    "an expert customer service rep for the housing collective HOME0001.",
)
tools = [tool]

In [4]:
tool.invoke("what is home0001?")

'Home0001 is a global housing network. Each 0001 home is fully-equipped and furnished. Move in with just your suitcase. Swap cities whenever you like.\n\nHome0001 is a distributed housing collective: in addition to community dinners and events, homeowners get access to 0001 homes in other cities for free. No nightly rate; just a cleaning fee each time. Own one home; live flexibly between multiple locations.\n\nHome0001 is initiated by a multi-disciplinary collective working across art, architecture, technology, and design, and currently based in los angeles, new york, paris, berlin, and london. Designed together with world renowned architects, 0001 homes are fully equipped and furnished and are part of an expanding network.\n\nHome0001 is a distributed housing collective: in addition to community dinners and events, homeowners get access to 0001 homes in other cities for free. No nightly rate; just a cleaning fee each time. Own one home, live in many places. '

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

memory = MemorySaver()

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [8]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "abc123"}}

for event in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! I'm bob
================================== Ai Message ==================================

Hello Bob! How can I assist you today?


In [10]:
query = "What is Home0001?"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is Home0001?
================================== Ai Message ==================================
Tool Calls:
  FAQ_retriever (call_6sD0vJc2hUVd5DmhDxxHFWsR)
 Call ID: call_6sD0vJc2hUVd5DmhDxxHFWsR
  Args:
    query: What is Home0001?
================================= Tool Message =================================
Name: FAQ_retriever

Home0001 is a global housing network. Each 0001 home is fully-equipped and furnished. Move in with just your suitcase. Swap cities whenever you like.

Home0001 is a distributed housing collective: in addition to community dinners and events, homeowners get access to 0001 homes in other cities for free. No nightly rate; just a cleaning fee each time. Own one home; live flexibly between multiple locations.

Home0001 is a distributed housing collective: in addition to community dinners and events, homeowners get access to 0001 homes in other cities for free. No nightly rate; j

In [11]:
query = "i am michaelangelo. can i buy home001 in italy?"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

i am michaelangelo. can i buy home001 in italy?
================================== Ai Message ==================================
Tool Calls:
  FAQ_retriever (call_dK8ZyVEdneBfQ2H1soevG7Zh)
 Call ID: call_dK8ZyVEdneBfQ2H1soevG7Zh
  Args:
    query: Can I buy Home0001 in Italy?
================================= Tool Message =================================
Name: FAQ_retriever

Home0001 is a global housing network. Each 0001 home is fully-equipped and furnished. Move in with just your suitcase. Swap cities whenever you like.

Homebuyers purchasing an 0001 home in los angeles get free access to 0001 homes in other cities too, while making their home available to other members of the collective when they’re away. Home0001 organizes the practicalities and keeps additional homes in each location for greater flexibility.

When you buy a 0001 home, you own the title on your individual home in the traditional way 

In [12]:
query = "what's my name? can i buy home001 in spain?"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what's my name? can i buy home001 in spain?
================================== Ai Message ==================================
Tool Calls:
  FAQ_retriever (call_m8xl6Xf1hw26uPDOGvZRadRv)
 Call ID: call_m8xl6Xf1hw26uPDOGvZRadRv
  Args:
    query: Can I buy Home0001 in Spain?
================================= Tool Message =================================
Name: FAQ_retriever

Home0001 is a global housing network. Each 0001 home is fully-equipped and furnished. Move in with just your suitcase. Swap cities whenever you like.

Homebuyers purchasing an 0001 home in los angeles get free access to 0001 homes in other cities too, while making their home available to other members of the collective when they’re away. Home0001 organizes the practicalities and keeps additional homes in each location for greater flexibility.

When you buy a 0001 home, you own the title on your individual home in the traditional way (so 